In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


data_path = "client_14.pt"
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Client 14's dataset {data_path} not found!")

client_data = torch.load(data_path)
client_train_data, client_train_targets = client_data['data'], client_data['targets']

# DataLoader for Client 14's Training Set
batch_size = 128
client_train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(client_train_data, client_train_targets),
    batch_size=batch_size, shuffle=True
)


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        x = self.conv1(x)
        x = self.bn1(x)
        out = F.relu(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        feature = out.view(out.size(0), -1)
        out = self.linear(feature)

        if return_features:
            return out, feature
        else:
            return out


def resnet18(num_classes=100):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)


def resnet34(num_classes=10):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes)


def resnet50(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)


def resnet101(num_classes=10):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes)


def resnet152(num_classes=10):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes)


model_filenames = [
# based on the precious stage, put round 2's Top-K model here 
]

models = []
for filename in model_filenames:
    checkpoint_path = f'./checkpoint/{filename}'
    if not os.path.exists(checkpoint_path):
        print(f"Warning: Teacher model checkpoint {checkpoint_path} not found. Skipping.")
        continue
    model = resnet18(num_classes=100).to(device)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    # Adjust the key based on how the checkpoint was saved
    if 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    elif 'net' in checkpoint:
        model.load_state_dict(checkpoint['net'])
    else:
        model.load_state_dict(checkpoint)  # Assume entire state_dict
    model.eval()
    models.append(model)

if not models:
    raise ValueError("No teacher models loaded. Ensure that model_filenames are correctly populated and checkpoints exist.")


class MetaModel(nn.Module):
    def __init__(self, num_models):
        super(MetaModel, self).__init__()
        # Initialize weights equally
        self.weights = nn.Parameter(torch.ones(num_models) / num_models)

    def forward(self, outputs):
        """
        Combine teacher model outputs using learnable weights.
        
        Args:
            outputs (torch.Tensor): Tensor of shape [num_models, batch_size, num_classes]
        
        Returns:
            torch.Tensor: Combined output of shape [batch_size, num_classes]
        """
        # Apply softmax to weights
        normalized_weights = F.softmax(self.weights, dim=0).view(-1, 1, 1)  # Shape: [num_models, 1, 1]
        # Apply weights to teacher outputs
        weighted_outputs = normalized_weights * outputs  # Broadcasting weights
        # Sum weighted outputs to get combined output
        combined_output = torch.sum(weighted_outputs, dim=0)  # Shape: [batch_size, num_classes]
        return combined_output


meta_model = MetaModel(num_models=len(models)).to(device)
meta_checkpoint_path = './weight_checkpointsdir/meta_weights_client14.pth'
if not os.path.exists(meta_checkpoint_path):
    raise FileNotFoundError(f"MetaModel checkpoint {meta_checkpoint_path} not found!")

meta_checkpoint = torch.load(meta_checkpoint_path, map_location=device)
# Adjust the key based on how the checkpoint was saved
if 'state_dict' in meta_checkpoint:
    meta_model.load_state_dict(meta_checkpoint['state_dict'])
elif 'net' in meta_checkpoint:
    meta_model.load_state_dict(meta_checkpoint['net'])
else:
    meta_model.load_state_dict(meta_checkpoint)  # Assume entire state_dict

meta_model.eval()


local_model_checkpoint_path = './checkpoint/cifar10014.pth'
if not os.path.exists(local_model_checkpoint_path):
    raise FileNotFoundError(f"Local model checkpoint {local_model_checkpoint_path} not found!")

local_model = resnet18(num_classes=100).to(device)
local_checkpoint = torch.load(local_model_checkpoint_path, map_location=device)
# Adjust the key based on how the checkpoint was saved
if 'state_dict' in local_checkpoint:
    local_model.load_state_dict(local_checkpoint['state_dict'])
elif 'net' in local_checkpoint:
    local_model.load_state_dict(local_checkpoint['net'])
else:
    local_model.load_state_dict(local_checkpoint)  # Assume entire state_dict

local_model.eval()


student_model = resnet18(num_classes=100).to(device)
student_model.load_state_dict(local_model.state_dict())  # Initialize with local model's parameters
student_model.train()  # Set to training mode


def distillation_loss(student_outputs, teacher_outputs, labels, temperature, lambda_reg, student_model, local_model):
    """
    Compute the knowledge distillation loss.
    
    Args:
        student_outputs (torch.Tensor): Logits from the student model.
        teacher_outputs (torch.Tensor): Logits from the teacher (combined) model.
        labels (torch.Tensor): Ground truth labels.
        temperature (float): Temperature parameter for softening probabilities.
        lambda_reg (float): Regularization strength.
        student_model (nn.Module): The student model.
        local_model (nn.Module): The local (pre-distilled) model.
    
    Returns:
        torch.Tensor: Combined loss value.
    """
    # Soft targets
    soft_teacher = F.softmax(teacher_outputs / temperature, dim=1)  # Correct usage
    soft_student = F.log_softmax(student_outputs / temperature, dim=1)  # Correct usage

    # KL Divergence loss (soft-label matching)
    kd_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temperature ** 2)

    # Cross-Entropy loss (hard labels)
    ce_loss = F.cross_entropy(student_outputs, labels)

    # L2 Regularization on model parameters
    l2_reg = 0.0
    for param_student, param_local in zip(student_model.parameters(), local_model.parameters()):
        l2_reg += torch.norm(param_student - param_local, p=2)
    l2_reg = lambda_reg * l2_reg

    return kd_loss + ce_loss + l2_reg


def train_student_model(student_model, teacher_models, meta_model, local_model, train_loader, optimizer, num_epochs=200, T=2.0, lambda_reg=0.01, patience=20):
    """
    Train the student model using knowledge distillation with regularization.
    
    Args:
        student_model (nn.Module): The student model to be trained.
        teacher_models (list of nn.Module): List of teacher models.
        meta_model (nn.Module): The MetaModel that combines teacher outputs.
        local_model (nn.Module): The local (pre-distilled) model.
        train_loader (DataLoader): DataLoader for training data.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        num_epochs (int): Maximum number of training epochs.
        T (float): Temperature parameter for KD.
        lambda_reg (float): Regularization strength.
        patience (int): Number of epochs to wait for improvement before early stopping.
    
    Returns:
        None
    """
    best_acc = 0.0
    patience_counter = 0

    # Ensure checkpoint directory exists
    checkpoint_dir = './student_checkpointsdir/'
    os.makedirs(checkpoint_dir, exist_ok=True)

    for epoch in range(num_epochs):
        student_model.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Get teacher outputs within no_grad to prevent gradient tracking
            with torch.no_grad():
                teacher_outputs = torch.stack([model(inputs) for model in teacher_models], dim=0)  # Shape: [num_teachers, batch_size, num_classes]
                teacher_combined_outputs = meta_model(teacher_outputs)  # Shape: [batch_size, num_classes]

            # Get student outputs
            student_outputs = student_model(inputs)

            # Compute knowledge distillation loss
            loss = distillation_loss(student_outputs, teacher_combined_outputs, labels, T, lambda_reg, student_model, local_model)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(student_outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = total_loss / len(train_loader.dataset)
        train_acc = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%')

        # Early stopping check
        if train_acc > best_acc:
            torch.save(student_model.state_dict(), os.path.join(checkpoint_dir, 'student_model_client14.pth'))
            best_acc = train_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break


# Initialize optimizer with weight decay for regularization
optimizer = optim.SGD(student_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

# Start training
train_student_model(
    student_model=student_model,
    teacher_models=models,
    meta_model=meta_model,
    local_model=local_model,
    train_loader=client_train_loader,
    optimizer=optimizer,
    num_epochs=200,
    T=2.0,
    lambda_reg=0.01,
    patience=20
)


def evaluate_model(model, data_loader, criterion):
    """
    Evaluate the model's performance on a given dataset.
    
    Args:
        model (nn.Module): The model to evaluate.
        data_loader (DataLoader): DataLoader for evaluation data.
        criterion (nn.Module): Loss function.
    
    Returns:
        tuple: (average_loss, accuracy)
    """
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = 100 * correct / total
    return avg_loss, accuracy


test_data_path = "client_14_test.pt"
if not os.path.exists(test_data_path):
    raise FileNotFoundError(f"Client 14's test dataset {test_data_path} not found!")

client_test_data = torch.load(test_data_path)
client_test_inputs, client_test_labels = client_test_data['data'], client_test_data['targets']

# DataLoader for Client 14's Test Set
client_test_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(client_test_inputs, client_test_labels),
    batch_size=batch_size, shuffle=False
)


# Define loss criterion for evaluation
evaluation_criterion = nn.CrossEntropyLoss()

# Load Best Student Model
checkpoint_dir = './student_checkpointsdir/'
best_student_path = os.path.join(checkpoint_dir, 'student_model_client14.pth')
if not os.path.exists(best_student_path):
    raise FileNotFoundError(f"Best student model checkpoint {best_student_path} not found!")

student_model.load_state_dict(torch.load(best_student_path, map_location=device))
student_model.eval()

# Evaluate on Test Set
test_loss, test_acc = evaluate_model(student_model, client_test_loader, evaluation_criterion)
print(f'Test Loss: {test_loss:.3f} | Test Accuracy: {test_acc:.2f}%')
